In [152]:
#https://jasdeep06.github.io/posts/Understanding-LSTM-in-Tensorflow-MNIST/

In [153]:
import tensorflow as tf

In [154]:
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets('C:/test/tensorflow/mnist/input_data', one_hot=True)

In [155]:
#unrolled through 28 time steps
time_steps = 8
#hidden LSTM units
num_units = 128
#rows of 28 pixels
n_input = 8
#learning rate for adam
learning_rate = 0.001
#mnist is meant to be classified in 10 classes(0-9).
#n_classes=10
#size of batch
batch_size = 16

test_rate = 0.1
h_size = time_steps * n_input
API_padding = True

In [156]:
%run ../MotifAnalysis/Hooklog3.ipynb
Hooklog = Hooklog3

In [157]:
# old data
'''
in_base_dir = "C:/test/tensorflow/hooklog/"
in_directories = ["allaple_woj_g_98_year2017", # 16k
                  "somoto_woj_year2017", # 17k
                  "morstar_g_54", # 85k
                  "bettersurf_woj_g_137+", # 106k
                  "elkern_woj_g_127", # 44k
                  "lydra_woj_g_44", # 30k
                  "mira_woj_g_107", # 14k
                  "sytro_woj_g_166" # 34k
                 ]

# new data
in_base_dir = "C:/Users/hsiao/Downloads/GitHub/Tensorflow/hooklogs/"
in_directories = [
    "allaple_woj_g_98_year2017",
    "bettersurf_woj_g_137+",
    "elkern_woj_g_127",
    "fakeav_g_132",
    "loring_g_15",
    "morstar_g_54",
    "rahack_g_39",
    "sytro_woj_g_166"
]
'''

# yj
in_base_dir = "C:/test/hooklog_family_yj/hooklog/"
in_directories = None
if in_directories is None:
    import os
    in_directories = next(os.walk(in_base_dir))[1]

# real dirs
in_directories = [in_base_dir + d + '/' for d in in_directories]

In [158]:
in_parseFirstPar = False

#in_apifreq_dir = "C:/Users/hsiao/Dropbox/notebook/MotifAnalysis/pickle/"
#in_apifreq_dir = "C:/Users/hsiao/Downloads/GitHub/Tensorflow/pickles/"
in_apifreq_dir = "C:/test/hooklog_family_yj/pickle/"

in_apifreq_dicts = [in_apifreq_dir+"apifreq_dict_"+d.split("/")[-2]+".pickle" for d in in_directories]

n_classes = len(in_directories)
classnames = list(map(lambda x: x.split("/")[-2], in_directories))
print(n_classes, classnames)

9 ['10.downloa', '13.strictor', '16.vobfus', '17.mplug', '19.solimba', '21.autoit', '23.expiro', '27.allaple', '35.rahack']


In [159]:
import pickle

apifreq_dict = dict()
_total = 0
for pkf in in_apifreq_dicts:
    with open(pkf, 'rb') as f:
        this_dict = pickle.load(f)
        for k in this_dict:
            if k in apifreq_dict:
                apifreq_dict[k] += this_dict[k]
            else:
                apifreq_dict[k] = this_dict[k]
            _total += this_dict[k]

s_dict = {item[0]: item for item in [(k, i, i/(len(apifreq_dict)-1), apifreq_dict[k], apifreq_dict[k]/_total) for i, k in enumerate(sorted(apifreq_dict, key = apifreq_dict.get, reverse=False))]}
print(s_dict)

{'TerminateProcess': ('TerminateProcess', 0, 0.0, 8, 0.0001410288051334485), 'GetUrlCacheEntryInfo': ('GetUrlCacheEntryInfo', 1, 0.058823529411764705, 29, 0.0005112294186087508), 'CopyFile': ('CopyFile', 2, 0.11764705882352941, 51, 0.0008990586327257342), 'InternetConnect': ('InternetConnect', 3, 0.17647058823529413, 54, 0.0009519444346507774), 'HttpSendRequest': ('HttpSendRequest', 4, 0.23529411764705882, 54, 0.0009519444346507774), 'InternetOpen': ('InternetOpen', 5, 0.29411764705882354, 57, 0.0010048302365758205), 'ExitProcess': ('ExitProcess', 6, 0.35294117647058826, 65, 0.0011458590417092692), 'CreateProcess': ('CreateProcess', 7, 0.4117647058823529, 151, 0.0026619186968938405), 'CreateThread': ('CreateThread', 8, 0.47058823529411764, 152, 0.002679547297535522), 'RegDeleteKey': ('RegDeleteKey', 9, 0.5294117647058824, 175, 0.003085005112294186), 'CreateProcessInternal': ('CreateProcessInternal', 10, 0.5882352941176471, 299, 0.005270951591862638), 'DeleteFile': ('DeleteFile', 11, 0.

In [160]:
import os
import random

xx_train_list = list()
yy_train_list = list()

xx_test_list = list()
yy_test_list = list()

train_name_list = list()
test_name_list = list()

name_dict = dict()

for label, in_dir in enumerate(in_directories):
    hl_list = next(os.walk(in_dir))[2] # get all filenames in the in_directory
    hl_list = [os.path.join(in_dir, f) for f in hl_list] # filepathname list
    hl_list = list(filter(lambda f: f.endswith(".hooklog"), hl_list)) # in case some non-hooklog file in the folder
    
    #shuffle list
    random.shuffle(hl_list) # <-------------- random here
    test_size = int(len(hl_list)*test_rate)

    for i, file in enumerate(hl_list):
        hl3 = Hooklog(file, in_parseFirstPar)
        li_li = list() # for hacking moving_window # 20171129
        
        for start in range(0, len(hl3.li), h_size):
            end = start + h_size
            #print(hl3.digitname, start, end)
            
            li = list()
            for (t, api) in hl3.li[start:end]:
                li.append(s_dict[api][4]) # <-- encode
            # hack
            if len(li) < h_size:
                #print("!!! change img_cols to a smaller number", len(hl3.li))
                #print(hl3.digitname, "has smaller size", len(hl3.li), "need img_cols", str(img_cols))
                if API_padding:
                    #print("padding 1.0", str(img_cols - len(hl3.li)))
                    for _ in range(h_size - len(li)):
                        li.append(1.0)
            li_li.append(li)

            if(i < test_size):
                xx_test_list.extend([li])
                a = [0] * n_classes
                a[label] = 1
                yy_test_list.extend([a])
                test_name_list.append((hl3.digitname, start))
                name_dict[hl3.digitname] = label
            else:
                xx_train_list.extend([li])
                a = [0] * n_classes
                a[label] = 1
                yy_train_list.extend([a])
                train_name_list.append((hl3.digitname, start))
                name_dict[hl3.digitname] = label
                
            break; # stop moving window!! Must break!

In [161]:
import numpy as np

#x_train = np.ndarray(shape = (len(xx_train_list), img_rows, img_cols), buffer = np.array(xx_train_list))
#y_train = np.array(yy_train_list)
x_train = np.ndarray(shape = (len(xx_train_list), time_steps * n_input), buffer = np.array(xx_train_list))
y_train = np.array(yy_train_list)

print(x_train.shape)
print(y_train.shape)

(173, 64)
(173, 9)


In [162]:
#x_test = x_train.copy()
#y_test = y_train.copy()

x_test= np.ndarray(shape = (len(xx_test_list), time_steps * n_input), buffer = np.array(xx_test_list))
y_test = np.array(yy_test_list)

print(x_test.shape)
print(y_test.shape)

(15, 64)
(15, 9)


### TF

In [163]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

In [164]:
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

In [165]:
#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input_x=tf.unstack(x, time_steps, 1)

In [166]:
#defining the network
with tf.variable_scope('anystring', reuse = tf.AUTO_REUSE):
    lstm_layer=tf.contrib.rnn.BasicLSTMCell(num_units,forget_bias=1)
    outputs,_=tf.nn.static_rnn(lstm_layer,input_x,dtype="float32") # Mike: I change it to tf.nn.
    
    #converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
    prediction=tf.matmul(outputs[-1],out_weights)+out_bias
    
    
    #loss_function
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    #optimization
    opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    #model evaluation
    correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [167]:
def my_next_batch(x_test, y_test, seq, start, batch_size):
    end = start + batch_size
    if end > len(x_test):
        # Shuffle the data
        perm = np.arange(len(x_test))
        np.random.shuffle(perm)
        # Start next epoch
        start = 0
        end = batch_size
        seq = perm
    return x_test[seq][start:end], y_test[seq][start:end], seq, end

In [168]:
#initialize variables
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
epoch = 1

b = 0
seq = np.arange(len(x_train))

while epoch < 500:
    #batch_x,batch_y=mnist.train.next_batch(batch_size=batch_size)
    batch_x, batch_y, seq, b = my_next_batch(x_train, y_train, seq, b, batch_size)
    batch_x = batch_x.reshape((batch_size, time_steps, n_input))
    #print(type(batch_x), batch_x.shape, type(batch_y), batch_y.shape)

    sess.run(opt, feed_dict={x: batch_x, y: batch_y})

    if epoch %10 == 0:
        acc = sess.run(accuracy, feed_dict = {x: batch_x, y: batch_y})
        los = sess.run(loss, feed_dict = {x: batch_x, y: batch_y})
        print("For epoch:", epoch, "Accuracy:", acc, "Loss:",los)
        #if acc >= 0.95: break # hack

    epoch += 1

For epoch: 10 Accuracy: 0.0 Loss: 4.57935
For epoch: 20 Accuracy: 0.6875 Loss: 2.01232
For epoch: 30 Accuracy: 0.125 Loss: 2.05776
For epoch: 40 Accuracy: 0.3125 Loss: 2.04054
For epoch: 50 Accuracy: 0.25 Loss: 1.91506
For epoch: 60 Accuracy: 0.625 Loss: 1.53506
For epoch: 70 Accuracy: 0.625 Loss: 1.37525
For epoch: 80 Accuracy: 0.625 Loss: 1.27842
For epoch: 90 Accuracy: 0.5625 Loss: 1.20963
For epoch: 100 Accuracy: 0.6875 Loss: 0.917984
For epoch: 110 Accuracy: 0.6875 Loss: 0.712812
For epoch: 120 Accuracy: 0.6875 Loss: 0.90244
For epoch: 130 Accuracy: 0.875 Loss: 0.544405
For epoch: 140 Accuracy: 0.8125 Loss: 0.583215
For epoch: 150 Accuracy: 0.8125 Loss: 0.491002
For epoch: 160 Accuracy: 0.875 Loss: 0.462655
For epoch: 170 Accuracy: 0.875 Loss: 0.464364
For epoch: 180 Accuracy: 0.875 Loss: 0.389735
For epoch: 190 Accuracy: 0.8125 Loss: 0.524883
For epoch: 200 Accuracy: 0.875 Loss: 0.368602
For epoch: 210 Accuracy: 0.75 Loss: 0.625343
For epoch: 220 Accuracy: 0.8125 Loss: 0.458657
F

In [169]:
#calculating test accuracy
#test_data = mnist.test.images[:128].reshape((-1, time_steps, n_input))
#test_label = mnist.test.labels[:128]
test_data = x_test.reshape((-1, time_steps, n_input))
test_label = y_test
print("Testing Accuracy:", sess.run(accuracy, feed_dict = {x: test_data, y: test_label}))

Testing Accuracy: 0.933333


In [170]:
vs = tf.trainable_variables()
print('There are', len(vs), 'train_able_variables in the Graph')
for v in vs:
    print(v)

There are 20 train_able_variables in the Graph
<tf.Variable 'Variable:0' shape=(128, 9) dtype=float32_ref>
<tf.Variable 'Variable_1:0' shape=(9,) dtype=float32_ref>
<tf.Variable 'rnn/basic_lstm_cell/kernel:0' shape=(136, 512) dtype=float32_ref>
<tf.Variable 'rnn/basic_lstm_cell/bias:0' shape=(512,) dtype=float32_ref>
<tf.Variable 'forward/rnn/basic_lstm_cell/kernel:0' shape=(136, 512) dtype=float32_ref>
<tf.Variable 'forward/rnn/basic_lstm_cell/bias:0' shape=(512,) dtype=float32_ref>
<tf.Variable 'anystring/rnn/basic_lstm_cell/kernel:0' shape=(136, 512) dtype=float32_ref>
<tf.Variable 'anystring/rnn/basic_lstm_cell/bias:0' shape=(512,) dtype=float32_ref>
<tf.Variable 'Variable_2:0' shape=(128, 9) dtype=float32_ref>
<tf.Variable 'Variable_3:0' shape=(9,) dtype=float32_ref>
<tf.Variable 'Variable_4:0' shape=(128, 9) dtype=float32_ref>
<tf.Variable 'Variable_5:0' shape=(9,) dtype=float32_ref>
<tf.Variable 'Variable_6:0' shape=(128, 9) dtype=float32_ref>
<tf.Variable 'Variable_7:0' shape=(